<a href="https://colab.research.google.com/github/sf-rahul/Tensorflow/blob/master/Recurrent_nueral_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**time series forecasting using rnns.**

In [0]:
#rnnn
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

tf.random.set_seed(1)

In [0]:
#function to create time series data of length n_steps
#input to the rnn is [batch_size, time steps , dimensionality]
#a time series is cosistting of values having per time step
#value is univariabte then it is calles univariate time series
#value is multivariate then it is calles multivariate time series.
def generate_time_series(batch_size,n_steps):
  freq1,freq2,offset1,offset2 = np.random.rand(4,batch_size,1)#4 random array of shape (batch_size,1)
  time = np.linspace(0,1,n_steps)
  series = 0.5  * np.sin((time-offset1) * (freq1*10+10)) #wave1
  series+= 0.2 * np.sin((time-offset2) * (freq2*20 +20)) #+wave2
  series +=0.1*(np.random.rand(batch_size,n_steps) - 0.5) #+noise
  #print(series)
  #to add new axis to data.
  return series[...,np.newaxis].astype(np.float32)




In [0]:
series = generate_time_series(4,3) #shape(4,2,1)
print(series)
print('--------------')
x,y=series[:3,:2],series[:3,-1]
print(x)
print('------------------')
print(y)


[[[-0.09269179]
  [-0.27487454]
  [ 0.26764813]]

 [[-0.3955232 ]
  [-0.47525486]
  [-0.3857966 ]]

 [[ 0.58213425]
  [ 0.49402696]
  [ 0.30883703]]

 [[ 0.3142075 ]
  [ 0.546498  ]
  [ 0.05040054]]]
--------------
[[[-0.09269179]
  [-0.27487454]]

 [[-0.3955232 ]
  [-0.47525486]]

 [[ 0.58213425]
  [ 0.49402696]]]
------------------
[[ 0.26764813]
 [-0.3857966 ]
 [ 0.30883703]]


In [0]:
#creating training , validation and test sets
n_steps =50 #no of time stamps or steps  in a time series 
series = generate_time_series(10000,n_steps+1)
X_train ,y_train = series[:7000,:n_steps],series[:7000,-1]
X_valid,y_valid = series[7000:9000,:n_steps],series[7000:9000,-1]
X_test,y_test = series[9000:,:n_steps],series[9000:,-1]


In [4]:
print(X_train.shape,y_train.shape)
print(X_valid.shape,y_valid.shape)
print(X_test.shape,y_test.shape)

(7000, 50, 1) (7000, 1)
(2000, 50, 1) (2000, 1)
(1000, 50, 1) (1000, 1)


In [0]:
#Baseline model 1. 
#naive forecasting := predict the last value in each series.
#just pick the last value as predicted value.
y_pred = X_valid[:,-1]
np.mean(tf.keras.losses.mean_squared_error(y_valid,y_pred))

0.020904407

In [0]:
#Baseline model 2
#use a fully connected network.
#the input will be flattene to 50*1 = 50 features(time steps)
#predictin will be simply linear compination of the time step values.
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=[50,1]),
                                     tf.keras.layers.Dense(1)])



In [0]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.compile(optimizer='adam', loss='mse')

In [0]:
model2.fit(x=X_train,y=y_train,epochs=20,validation_data=(X_valid,y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 1s 105us/sample - loss: 0.0883 - val_loss: 0.0373
Epoch 2/20
7000/7000 [==============================] - 0s 49us/sample - loss: 0.0255 - val_loss: 0.0185
Epoch 3/20
7000/7000 [==============================] - 0s 50us/sample - loss: 0.0152 - val_loss: 0.0129
Epoch 4/20
7000/7000 [==============================] - 0s 50us/sample - loss: 0.0116 - val_loss: 0.0105
Epoch 5/20
7000/7000 [==============================] - 0s 49us/sample - loss: 0.0097 - val_loss: 0.0091
Epoch 6/20
7000/7000 [==============================] - 0s 50us/sample - loss: 0.0083 - val_loss: 0.0079
Epoch 7/20
7000/7000 [==============================] - 0s 48us/sample - loss: 0.0073 - val_loss: 0.0070
Epoch 8/20
7000/7000 [==============================] - 0s 46us/sample - loss: 0.0065 - val_loss: 0.0062
Epoch 9/20
7000/7000 [==============================] - 0s 48us/sample - loss: 0.0058 - val_loss: 0.0056
Epoch 

**IMPLEMENTING SIMPLE RNN**

In [0]:
SimpleRnn = tf.keras.models.Sequential([
                                        tf.keras.layers.SimpleRNN(1,input_shape=[None,1])
                       ])

In [0]:
SimpleRnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
SimpleRnn.compile(optimizer='adam',loss='mse')
SimpleRnn.fit(x=X_train,y=y_train,epochs=20,validation_data=(X_valid,y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 3s 387us/sample - loss: 0.2219 - val_loss: 0.2026
Epoch 2/20
7000/7000 [==============================] - 2s 281us/sample - loss: 0.1850 - val_loss: 0.1682
Epoch 3/20
7000/7000 [==============================] - 2s 278us/sample - loss: 0.1554 - val_loss: 0.1413
Epoch 4/20
7000/7000 [==============================] - 2s 283us/sample - loss: 0.1316 - val_loss: 0.1186
Epoch 5/20
7000/7000 [==============================] - 2s 285us/sample - loss: 0.1089 - val_loss: 0.0963
Epoch 6/20
7000/7000 [==============================] - 2s 286us/sample - loss: 0.0875 - val_loss: 0.0769
Epoch 7/20
7000/7000 [==============================] - 2s 283us/sample - loss: 0.0690 - val_loss: 0.0601
Epoch 8/20
7000/7000 [==============================] - 2s 284us/sample - loss: 0.0533 - val_loss: 0.0459
Epoch 9/20
7000/7000 [==============================] - 2s 281us/sample - loss: 0.0402 - val_loss: 0.034

In [0]:
#The simple rnn is better than the naive one but not than the fully connected one.

**DEEP RNNS**
Stack multiple layers of the cells.

In [0]:
DeepRnn = tf.keras.models.Sequential([
                                 tf.keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                                 tf.keras.layers.SimpleRNN(20,return_sequences=True),
                                 tf.keras.layers.SimpleRNN(1)

 ])

In [0]:
DeepRnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, None, 20)          440       
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 20)          820       
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 1)                 22        
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________


In [0]:
DeepRnn.compile(optimizer='adam',loss='mse')
DeepRnn.fit(x=X_train,y=y_train,epochs=20,validation_data=(X_valid,y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0189 - val_loss: 0.0052
Epoch 2/20
7000/7000 [==============================] - 7s 930us/sample - loss: 0.0042 - val_loss: 0.0038
Epoch 3/20
7000/7000 [==============================] - 6s 921us/sample - loss: 0.0034 - val_loss: 0.0034
Epoch 4/20
7000/7000 [==============================] - 6s 908us/sample - loss: 0.0031 - val_loss: 0.0032
Epoch 5/20
7000/7000 [==============================] - 6s 907us/sample - loss: 0.0030 - val_loss: 0.0031
Epoch 6/20
7000/7000 [==============================] - 6s 907us/sample - loss: 0.0029 - val_loss: 0.0031
Epoch 7/20
7000/7000 [==============================] - 6s 918us/sample - loss: 0.0028 - val_loss: 0.0031
Epoch 8/20
7000/7000 [==============================] - 6s 914us/sample - loss: 0.0028 - val_loss: 0.0029
Epoch 9/20
7000/7000 [==============================] - 6s 924us/sample - loss: 0.0027 - val_loss: 0.0030


In [0]:
#A slight improvement in running time.
DeepRnn = tf.keras.models.Sequential([
                                 tf.keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                                 tf.keras.layers.SimpleRNN(20),
                                 tf.keras.layers.Dense(1)

 ])
DeepRnn.compile(optimizer='adam',loss='mse')


In [53]:
DeepRnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, None, 20)          440       
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 20)                820       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
_________________________________________________________________


In [54]:
DeepRnn.fit(x=X_train,y=y_train,epochs=20,validation_data=(X_valid,y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 6s 834us/sample - loss: 0.0248 - val_loss: 0.0051
Epoch 2/20
7000/7000 [==============================] - 4s 637us/sample - loss: 0.0040 - val_loss: 0.0035
Epoch 3/20
7000/7000 [==============================] - 5s 666us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 4/20
7000/7000 [==============================] - 5s 652us/sample - loss: 0.0032 - val_loss: 0.0032
Epoch 5/20
7000/7000 [==============================] - 5s 663us/sample - loss: 0.0030 - val_loss: 0.0032
Epoch 6/20
7000/7000 [==============================] - 5s 648us/sample - loss: 0.0030 - val_loss: 0.0031
Epoch 7/20
7000/7000 [==============================] - 4s 641us/sample - loss: 0.0030 - val_loss: 0.0029
Epoch 8/20
7000/7000 [==============================] - 5s 653us/sample - loss: 0.0029 - val_loss: 0.0031
Epoch 9/20
7000/7000 [==============================] - 4s 636us/sample - loss: 0.0029 - val_loss: 0.002

array([[[20, 21, 22]]])

FORECASTING SEREVERAL TIME STAMPS AHEAD

In [0]:
#TO PREDICT VAULE AT 10TH NEXT STEP JUST CHANGE TARGETS TO 10 STEP AHEAD.
#TO PREDICT THOSE NEXT STEP 10 VALUES.?
#SOLUTION 1:
series = generate_time_series(1,n_steps+10)
X_new ,y_new = series[:,:n_steps],series[:,n_steps:]
X=X_new
for step_ahead in range(10):
  y_pred_one= DeepRnn.predict(X[:,step_ahead:])[:,np.newaxis,:]
  X=np.concatenate([X,y_pred_one],axis=1)


Y_pred=X[:,n_steps:]




In [0]:
series = generate_time_series(1,n_steps+10)
X_new ,y_new = series[:,:n_steps],series[:,n_steps]

In [48]:
series[:,:n_steps].shape
y=(DeepRnn.predict(X_new[:,1:]))
y.shape

(1, 1)

In [56]:
np.mean(tf.keras.losses.mean_squared_error(y_new.reshape(10,1),Y_pred.reshape(10,1)))

0.0015100244

In [57]:
Y_pred.reshape(10,1)

array([[-0.27847916],
       [-0.26748604],
       [-0.24663723],
       [-0.21617058],
       [-0.17288312],
       [-0.10835671],
       [-0.01626887],
       [ 0.1005441 ],
       [ 0.23409542],
       [ 0.34945607]], dtype=float32)

In [58]:
y_new.reshape(10,1)

array([[-0.30047086],
       [-0.2178156 ],
       [-0.20377576],
       [-0.20663038],
       [-0.16348438],
       [-0.11320417],
       [-0.03477712],
       [ 0.08369689],
       [ 0.15609337],
       [ 0.29115686]], dtype=float32)

In [59]:
#method 2:
Y_pred.reshape(10,1) - y_new.reshape(10,1)



array([[ 0.0219917 ],
       [-0.04967044],
       [-0.04286146],
       [-0.0095402 ],
       [-0.00939874],
       [ 0.00484745],
       [ 0.01850825],
       [ 0.0168472 ],
       [ 0.07800205],
       [ 0.05829921]], dtype=float32)

Sequnce to verctor model.

In [0]:
n_steps = 50
series = generate_time_series(10000,n_steps+10)
X_train,Y_train = series[:7000,:n_steps],series[:7000,-10:,0]
X_valid,Y_valid = series[7000:9000,:n_steps],series[7000:9000,-10:,0]
X_test,Y_test=series[9000:,:n_steps],series[9000:,-10,0]

In [0]:
SeqtoVec = tf.keras.models.Sequential([tf.keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                                       tf.keras.layers.SimpleRNN(20),
                                       tf.keras.layers.Dense(10)])

In [21]:
SeqtoVec.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_5 (SimpleRNN)     (None, None, 20)          440       
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 20)                820       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
Total params: 1,470
Trainable params: 1,470
Non-trainable params: 0
_________________________________________________________________


In [22]:
SeqtoVec.compile(optimizer='adam',loss='mse')
SeqtoVec.fit(x=X_train,y=Y_train,epochs=20,validation_data=(X_valid,Y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 6s 916us/sample - loss: 0.0608 - val_loss: 0.0270
Epoch 2/20
7000/7000 [==============================] - 5s 676us/sample - loss: 0.0228 - val_loss: 0.0187
Epoch 3/20
7000/7000 [==============================] - 5s 679us/sample - loss: 0.0171 - val_loss: 0.0186
Epoch 4/20
7000/7000 [==============================] - 5s 705us/sample - loss: 0.0152 - val_loss: 0.0142
Epoch 5/20
7000/7000 [==============================] - 5s 704us/sample - loss: 0.0136 - val_loss: 0.0122
Epoch 6/20
7000/7000 [==============================] - 5s 685us/sample - loss: 0.0126 - val_loss: 0.0125
Epoch 7/20
7000/7000 [==============================] - 5s 697us/sample - loss: 0.0123 - val_loss: 0.0124
Epoch 8/20
7000/7000 [==============================] - 5s 694us/sample - loss: 0.0118 - val_loss: 0.0116
Epoch 9/20
7000/7000 [==============================] - 5s 698us/sample - loss: 0.0115 - val_loss: 0.010

In [18]:
Y_train.shape

(7000, 0, 1)

In [24]:
#predicting 10 values at once.
series1 = generate_time_series(1,n_steps+10)
X_new ,y_new = series1[:,:n_steps],series1[:,n_steps:]
X=X_new
Y_pred=SeqtoVec.predict(X)
Y_pred

array([[-0.4429071 , -0.34703997, -0.2647407 , -0.22561099, -0.20047702,
        -0.20758827, -0.21689215, -0.1747995 , -0.06746349,  0.03275465]],
      dtype=float32)

In [0]:
Sequence to Sequenct Model. SLight imporvement.